In [16]:
import pandas as pd
import tellurium as te
import antimony as at
import simplesbml 
import mira
import libsbml
import numpy as np
from mira.sources.sbml import template_model_from_sbml_file
from mira.metamodel import Distribution

import os
import pyciemss
import torch

import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

from pyciemss.integration_utils.intervention_builder import (
    param_value_objective,
    start_time_objective,
    start_time_param_value_objective,
)
import math

from mira.modeling.amr.petrinet import AMRPetriNetModel
from mira.modeling import Model

def convert_antimony_to_petri_net_model(antimony_str, sbml_file, corrected_sbml_file, petri_net_file, name='test_name', desc='test_desc', seed=0):
    
    np.random.seed(seed)

    # load antimony and save as SBML
    rr_model = te.loada(antimony_str)
    rr_model.exportToSBML(sbml_file)

    # correct SBML file to include IDs, initial_concentrations, etc.
    d = libsbml.readSBMLFromFile(sbml_file)
    m = d.getModel()
    for species in m.getListOfSpecies():
        species.name = species.id
        if math.isnan(species.initial_concentration):
            species.initial_concentration = species.initial_amount
        species.has_only_substance_units = False

    # for reaction in m.reactions:
    #     rate_law = reaction.getKineticLaw()
    #     print(reaction)
    #     print(rate_law)
    #     print(rate_law.formula)
    #     for parameter in rate_law.parameters:
    #         print('test')
    #         print(parameter, parameter.name, parameter.id, parameter.value)


    libsbml.writeSBMLToFile(d,corrected_sbml_file) 

    # create mira model from corrected SBML file and assign priors 
    mira_model = template_model_from_sbml_file(corrected_sbml_file)
    
    rate_constant_dist = Distribution(
                    type= "Uniform1",
                    parameters= {"minimum": 1e-3,"maximum": 1.0 }     
                     )
    for name,parameter in mira_model.parameters.items():
        if name.startswith('k'): # only assign priors for rate constants
            parameter.distribution = rate_constant_dist

    if not mira_model.templates:
        print("ERROR: No templates found in MIRA Model!")
    else:
        print("MIRA Model contains templates.")

    # create petri net model from mira model and save as json
    petrinet_model = AMRPetriNetModel(Model(mira_model))
    petrinet_model.to_json_file(
            petri_net_file,
            name=name,
            description=desc,
            indent=2,
        )
    return petrinet_model, mira_model

In [21]:
antimony_str = 'test_antimony_file.txt'
sbml_file = 'test1.xml'
corrected_sbml_file = 'test1_corrected.xml'
petri_net_file = 'test1.json'

test_petri_net_model, test_mira_model = convert_antimony_to_petri_net_model(antimony_str, sbml_file, corrected_sbml_file, petri_net_file)


MIRA Model contains templates.


In [ ]:

print(test_petri_net_model.states)
print(test_petri_net_model.transitions)
# Debug: Printing important MIRA model attributes
print("MIRA Model Parameters:", test_mira_model.parameters)
print("MIRA Model Templates:", test_mira_model.templates)
print("MIRA Model Initials:", test_mira_model.initials)
print("MIRA Model Observables:", test_mira_model.observables)

In [ ]:
test_mira_model.templates

In [ ]:
start_time = 0.0
end_time = 10.0
logging_step_size = 0.1
num_samples = 10
result1 = pyciemss.sample(petri_net_file, end_time, logging_step_size, num_samples, start_time=start_time)
display(result1['data'].head())

# Plot results for all states
schema = plots.trajectories(result1["data"], keep=".*_state")
plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, dpi=150)

In [13]:
antimony_str2 = './microbial_induced_calcium_carbonate_precipitation_model.txt'
sbml_file2 = 'test2.xml'
corrected_sbml_file2 = 'test2_corrected.xml'
petri_net_file2 = 'test2.json'

test_petri_net_model2, test_mira_model2  = convert_antimony_to_petri_net_model(antimony_str2, sbml_file2, corrected_sbml_file2, petri_net_file2)


<Reaction reaction_ureolysis>
<KineticLaw>
V_max_urease * urea / (K_m_urease + urea)
<Reaction reaction_protonation_NH3>
<KineticLaw>
k2f * pow(ammonia, 2) * pow(water, 2) - k2r * pow(ammonium, 2) * pow(hydroxide, 2)
<Reaction reaction_CO2_hydrolysis>
<KineticLaw>
K_cat_CA * CA_conc * carbon_dioxide / (K_m_CA + carbon_dioxide)
<Reaction reaction_dissociation_H2CO3>
<KineticLaw>
k4f * carbonic_acid * hydroxide - k4r * bicarbonate * water
<Reaction reaction_dissociation_HCO3>
<KineticLaw>
k5f * bicarbonate * hydroxide - k5r * carbonate * water
<Reaction reaction_CaCO3_precipitation>
<KineticLaw>
k6f * carbonate * calcium_ion - k6r * calcium_carbonate
ERROR: No templates found in MIRA Model!


In [ ]:


print(test_petri_net_model2.states)
print(test_petri_net_model2.transitions)
# Debug: Printing important MIRA model attributes
print("MIRA Model Parameters:", test_mira_model2.parameters)
print("MIRA Model Templates:", test_mira_model2.templates)
print("MIRA Model Initials:", test_mira_model2.initials)
print("MIRA Model Observables:", test_mira_model2.observables)


In [ ]:
result2 = pyciemss.sample(petri_net_file2, end_time, logging_step_size, num_samples, start_time=start_time)
display(result2['data'].head())

# Plot results for all states
schema2 = plots.trajectories(result2["data"], keep=".*_state")
plots.save_schema(schema2, "_schema.json")
plots.ipy_display(schema2, dpi=150)

In [17]:
antimony_str3 = './antimony_model_test3.txt'
sbml_file3 = 'test3.xml'
corrected_sbml_file3 = 'test3_corrected.xml'
petri_net_file3 = 'test3.json'

test_petri_net_model3, test_mira_model3  = convert_antimony_to_petri_net_model(antimony_str3, sbml_file3, corrected_sbml_file3, petri_net_file3)


print("MIRA Model Templates:", test_mira_model3.templates)

ERROR: No templates found in MIRA Model!
MIRA Model Templates: []


In [8]:
#!pip install pygraphviz
test_mira_model3.draw_graph('test3_graph')

In [18]:
print("MIRA Model Parameters:", test_mira_model3.parameters)
print("MIRA Model Templates:", test_mira_model3.templates)
print("MIRA Model Initials:", test_mira_model3.initials)
print("MIRA Model Observables:", test_mira_model3.observables)


MIRA Model Parameters: {'k1f': Parameter(name='k1f', display_name=None, description='', identifiers={}, context={}, units=None, value=1.0, distribution=Distribution(type='Uniform1', parameters={'minimum': 0.001, 'maximum': 1.0})), 'k1r': Parameter(name='k1r', display_name=None, description='', identifiers={}, context={}, units=None, value=1.0, distribution=Distribution(type='Uniform1', parameters={'minimum': 0.001, 'maximum': 1.0})), 'default_compartment': Parameter(name='default_compartment', display_name=None, description='', identifiers={}, context={}, units=None, value=1.0, distribution=None)}
MIRA Model Templates: []
MIRA Model Initials: {'S1': Initial(concept=Concept(name='S1', display_name='S1', description=None, identifiers={}, context={}, units=None), expression=1.0), 'S2': Initial(concept=Concept(name='S2', display_name='S2', description=None, identifiers={}, context={}, units=None), expression=1.0), 'S3': Initial(concept=Concept(name='S3', display_name='S3', description=None

In [1]:
import requests
import tellurium

from mira.sources.sbml import template_model_from_sbml_file
from mira.modeling import Model
from mira.modeling.ode import OdeModel
from mira.modeling.viz import GraphicalModel

ModuleNotFoundError: No module named 'mira'